In [ ]:
# Directory 
input_path = os.environ.get('input_path')
fig_output = os.environ.get('fig_output')
tab_output = os.environ.get('tab_output')

# Queries config
project_id_bq = os.environ.get('project_id_bq')
run_query = os.environ.get('run_query')

In [ ]:
# Merge between licitacao and licitacao_item
# Necessary to get the percentage of unique tenders that are both in licitacao and licitacao_item
# We use it to construct the variable 'Has item information'

def run_query_and_save_results():

    query = '''

    SELECT
        a.ano,
        a.id_municipio,
        a.sigla_uf,
        a.id_licitacao_bd,
        b.total_itens,
        CASE
            WHEN a.id_licitacao_bd IS NULL THEN 'RIGHT'
            WHEN b.id_licitacao_bd IS NULL THEN 'LEFT'
            ELSE 'BOTH'
        END AS join_type
    FROM (
        SELECT *
        FROM basedosdados-dev.world_wb_mides.licitacao
        WHERE id_licitacao_bd IS NOT NULL
      ) a
    FULL OUTER JOIN
        (SELECT
            id_licitacao_bd,
            COUNT(*) AS total_itens
        FROM
            basedosdados-dev.world_wb_mides.licitacao_item
        WHERE id_licitacao_bd IS NOT NULL
        GROUP BY
            id_licitacao_bd
        ) b
    ON
        a.id_licitacao_bd = b.id_licitacao_bd
        
    '''
    merge_licitacao_item = bd.read_sql(query, billing_project_id=project_id_bq)

    merge_licitacao_item.to_csv(os.path.join(input_path,'merge_licitacao_item.csv'), index=False, na_rep='', float_format='%.2f')

if __name__ == '__main__':
    if run_query == 'True':
        run_query_and_save_results()

In [ ]:
# Merge between licitacao and licitacao_participante
# Necessary to get the percentage of unique tenders that are both in licitacao and licitacao_participante
# We use it to construct the variable 'Has participant information'

def run_query_and_save_results():

    query = '''
    SELECT
        a.ano, a.id_municipio, a.sigla_uf, a.id_licitacao_bd,
        CASE
            WHEN a.id_licitacao_bd IS NULL THEN 'RIGHT'
            WHEN b.id_licitacao_bd IS NULL THEN 'LEFT'
            ELSE 'BOTH'
        END AS join_type
    FROM (
          SELECT *
          FROM basedosdados-dev.world_wb_mides.licitacao
          WHERE id_licitacao_bd IS NOT NULL
            ) a
    FULL OUTER JOIN
        (SELECT id_licitacao_bd
         FROM
            basedosdados-dev.world_wb_mides.licitacao_participante
         WHERE id_licitacao_bd IS NOT NULL
         GROUP BY
            id_licitacao_bd
        ) AS b
    ON
        a.id_licitacao_bd = b.id_licitacao_bd

    '''
    merge_licitacao_participante = bd.read_sql(query, billing_project_id=project_id_bq)

    merge_licitacao_participante.to_csv(os.path.join(input_path,'merge_licitacao_participante.csv'), index=False, na_rep='', float_format='%.2f')

if __name__ == '__main__':
    if run_query == 'True':
        run_query_and_save_results()

In [ ]:
# Counts of non-competitive, deserted and failed tenders
# We use it to construct the variables 'Non Competitive tenders', 'Deserted tenders' and 'Unsuccessful tenders'

def run_query_and_save_results():

    query = '''
    SELECT sigla_uf,
        COUNT(DISTINCT CASE WHEN (modalidade = '8' OR modalidade = '10') THEN id_licitacao_bd END) AS non_competitive_tender,
        COUNT(DISTINCT CASE WHEN situacao = '4' THEN id_licitacao_bd END) AS deserted_tender,
        COUNT(DISTINCT CASE WHEN situacao IN ('3', '5', '6') THEN id_licitacao_bd END) AS unsuccessful_tender,
    FROM basedosdados-dev.world_wb_mides.licitacao
    WHERE id_licitacao_bd IS NOT NULL
    GROUP BY sigla_uf

    '''

    licitacao_stats_uf = bd.read_sql(query, billing_project_id=project_id_bq)

    licitacao_stats_uf.to_csv(os.path.join(input_path,'licitacao_stats_uf.csv'), index=False, na_rep='', float_format='%.2f')

if __name__ == '__main__':
    if run_query == 'True':
        run_query_and_save_results()

In [ ]:
# Share of non-competitive tenders (by value)
# Non-competitive tenders are those with modalidade=8 or modalidade=10
# Winsorize to deal with outliers values

def run_query_and_save_results():

    query = '''

    WITH converted_data AS (
      SELECT
        sigla_uf,
        valor_corrigido,
        modalidade,
        id_licitacao_bd,
        SAFE_CAST(valor_corrigido AS FLOAT64) AS valor_corrigido_float
      FROM basedosdados-dev.world_wb_mides.licitacao
      WHERE id_licitacao_bd IS NOT NULL
    )
    SELECT
      sigla_uf,
      SUM(valor_corrigido_w) AS total_valor_corrigido,
      SUM(CASE WHEN (modalidade = '8' OR modalidade = '10') THEN valor_corrigido_w ELSE 0 END) / SUM(valor_corrigido_w) AS non_competitive_share_valor_corrigido
    FROM (
      SELECT
        sigla_uf,
        valor_corrigido_float AS valor_corrigido,
        modalidade,
        id_licitacao_bd,
        CASE
          WHEN valor_corrigido_float < percentile_lower THEN percentile_lower
          WHEN valor_corrigido_float > percentile_upper THEN percentile_upper
          ELSE valor_corrigido_float
        END AS valor_corrigido_w
      FROM (
        SELECT
          sigla_uf,
          valor_corrigido,
          modalidade,
          id_licitacao_bd,
          valor_corrigido_float,
          PERCENTILE_CONT(valor_corrigido_float, 0.001) OVER (PARTITION BY sigla_uf) AS percentile_lower,
          PERCENTILE_CONT(valor_corrigido_float, 0.999) OVER (PARTITION BY sigla_uf) AS percentile_upper
        FROM converted_data
      ) subquery
    ) w
    GROUP BY sigla_uf

    '''

    licitacao_share_valor_uf = bd.read_sql(query, billing_project_id=project_id_bq)

    licitacao_share_valor_uf.to_csv(os.path.join(input_path,'licitacao_share_valor_uf.csv'), index=False, na_rep='', float_format='%.2f')

if __name__ == '__main__':
    if run_query == 'True':
        run_query_and_save_results()

In [ ]:
# Counts licitacao_participante, group by tender
# Keep only competitive tenders
# We use it to construct the variables 'Avg. number of participants per tender' and 'Avg. number of suppliers per tender'

def run_query_and_save_results():

    query = '''
    SELECT lp.sigla_uf, lp.id_municipio, lp.id_licitacao_bd,
           COUNT(DISTINCT CONCAT(COALESCE(lp.documento, ''), COALESCE(lp.razao_social, ''))) AS participantes_distintos,
           COUNT(DISTINCT CASE WHEN lp.vencedor='1' THEN CONCAT(COALESCE(lp.documento, ''), COALESCE(lp.razao_social, '')) END) AS vencedores_distintos,
    FROM (
      SELECT sigla_uf, id_municipio, id_licitacao_bd, documento, razao_social, vencedor
      FROM basedosdados-dev.world_wb_mides.licitacao_participante
      WHERE NOT (documento IS NULL AND razao_social IS NULL) AND id_licitacao_bd IS NOT NULL
      ) lp
    INNER JOIN (
      SELECT id_licitacao_bd, modalidade
      FROM basedosdados-dev.world_wb_mides.licitacao
      WHERE id_licitacao_bd IS NOT NULL AND modalidade NOT IN ('8', '10')
      ) l
    ON lp.id_licitacao_bd = l.id_licitacao_bd
    GROUP BY lp.id_licitacao_bd, lp.id_municipio, lp.sigla_uf

    '''

    licitacao_participante_stats = bd.read_sql(query, billing_project_id=project_id_bq)

    licitacao_participante_stats.to_csv(os.path.join(input_path,'licitacao_participante_stats.csv'), index=False, na_rep='', float_format='%.2f')

if __name__ == '__main__':
    if run_query == 'True':
        run_query_and_save_results()

In [ ]:
# Counts licitacao_participante, group by state
# We use it to construct the variables 'Number of distinct participants', 'Number of distinct suppliers' and 'Firms among suppliers'

def run_query_and_save_results():

    query = '''
    SELECT sigla_uf, COUNT(DISTINCT CONCAT(COALESCE(documento, ''), COALESCE(razao_social, ''))) AS participantes_distintos,
        COUNT(DISTINCT CASE WHEN vencedor='1' THEN CONCAT(COALESCE(documento, ''), COALESCE(razao_social, '')) END) AS vencedores_distintos,
        COUNT(DISTINCT CASE WHEN tipo = '1' THEN CONCAT(COALESCE(documento, ''), COALESCE(razao_social, '')) END) AS firmas,
        COUNT(DISTINCT CASE WHEN (vencedor='1' AND tipo = '1') THEN CONCAT(COALESCE(documento, ''), COALESCE(razao_social, '')) END) AS firmas_vencedoras
    FROM basedosdados-dev.world_wb_mides.licitacao_participante
    WHERE NOT (documento IS NULL AND razao_social IS NULL) AND id_licitacao_bd IS NOT NULL
    GROUP BY sigla_uf
    '''

    licitacao_participante_stats_uf = bd.read_sql(query, billing_project_id=project_id_bq)

    licitacao_participante_stats_uf.to_csv(os.path.join(input_path,'licitacao_participante_stats_uf.csv'), index=False, na_rep='', float_format='%.2f')

if __name__ == '__main__':
    if run_query == 'True':
        run_query_and_save_results()

In [59]:
# Descriptive statistics table
# Join all variables

merge_licitacao_item = pd.read_csv(os.path.join(input_path, 'merge_licitacao_item.csv'))
merge_licitacao_participante = pd.read_csv(os.path.join(input_path,'merge_licitacao_participante.csv'))
licitacao_stats_uf = pd.read_csv(os.path.join(input_path,'licitacao_stats_uf.csv'))
licitacao_share_valor_uf = pd.read_csv(os.path.join(input_path,'licitacao_share_valor_uf.csv'))
licitacao_participante_stats = pd.read_csv(os.path.join(input_path,'licitacao_participante_stats.csv'))
licitacao_participante_stats_uf = pd.read_csv(os.path.join(input_path,'licitacao_participante_stats_uf.csv'))

# 1. Number of unique tenders and number of distinct municipalities
# id_licitacao_bd is the variable that uniquely identifies each row of the table licitacao
# From: merge_licitacao_item

tabela = merge_licitacao_item.groupby(['sigla_uf'])[['id_licitacao_bd', 'id_municipio']].nunique()
tabela.columns = ['Number of tenders', 'Number of distinct municipalities']

# 2. Tenders with item information ('Has item information')
# Identified by the quantity of tenders id (id_licitacao_bd) that are both in licitacao and licitacao_item
# From: merge_licitacao_item

tabela['Number of tenders with items'] = merge_licitacao_item[merge_licitacao_item['join_type'] == 'BOTH'].groupby('sigla_uf')['id_licitacao_bd'].nunique()
tabela['Number of tenders with items'] = tabela['Number of tenders with items'].fillna(0).astype(int)

tabela['Has item information'] = tabela['Number of tenders with items']/tabela['Number of tenders']

# 3. Tenders with participant information ('Has item information')
# Identified by the quantity of tenders id (id_licitacao_bd) that are both in licitacao and licitacao_participante
# From: merge_licitacao_participante

tabela['Number of tenders with participants'] = merge_licitacao_participante[merge_licitacao_participante['join_type'] == 'BOTH'].groupby('sigla_uf')['id_licitacao_bd'].nunique()
tabela['Has participant information'] = tabela['Number of tenders with participants']/tabela['Number of tenders']

# 4. Average number of items per tender
# From: merge_licitacao_item

tabela['Avg. number of items per tender'] = merge_licitacao_item[merge_licitacao_item['join_type'] == 'BOTH'].groupby('sigla_uf')['total_itens'].mean()
tabela['Avg. number of items per tender'] = tabela['Avg. number of items per tender'].astype(float)

# 5. Average number of participants and suppliers per tender
# From: licitacao_participante_stats

average_participants_suppliers = licitacao_participante_stats.groupby('sigla_uf')[['participantes_distintos','vencedores_distintos']].mean().reset_index()

average_participants_suppliers.rename({'participantes_distintos':'Avg. number of participants per tender',
                                       'vencedores_distintos':'Avg. number of suppliers per tender'},axis=1,inplace=True)

tabela = pd.merge(tabela, average_participants_suppliers, how='left', left_on='sigla_uf', right_on='sigla_uf')
tabela = pd.merge(tabela, licitacao_participante_stats_uf, how='left', left_on='sigla_uf', right_on='sigla_uf')

tabela.rename({'participantes_distintos':'Number of distinct participants',
               'vencedores_distintos':'Number of distinct suppliers',
               'firmas_vencedoras':'Number of suppliers firms'}, axis=1, inplace=True)

# 6. Percentage of firms among suppliers
# Firms: those with 14-digit document

tabela['Firms among suppliers'] = tabela['Number of suppliers firms']/tabela['Number of distinct suppliers']

# 7. Non-competitive tenders (number of tenders), Deserted and unsuccessful tenders
# From: licitacao_stats_uf

tabela = pd.merge(tabela, licitacao_stats_uf, how='left', left_on='sigla_uf', right_on='sigla_uf')

tabela['Non-competitive tenders'] = tabela['non_competitive_tender']/tabela['Number of tenders']
tabela['Deserted tenders'] = tabela['deserted_tender']/tabela['Number of tenders']
tabela['Unsuccessful tenders'] = tabela['unsuccessful_tender']/tabela['Number of tenders']

# 8. Non-competitive tenders (value of tenders)
# From: licitacao_share_valor_uf

tabela = pd.merge(tabela, licitacao_share_valor_uf, how='left', left_on='sigla_uf', right_on='sigla_uf')

tabela.rename({'non_competitive_share_valor_corrigido':'Non-competitive tenders value'}, axis=1, inplace=True)

# 9. Total
totais = tabela['Number of tenders'].astype(int).sum()
df_totais = pd.DataFrame({'Number of tenders': [totais]})
df_totais['Number of distinct municipalities'] = tabela['Number of distinct municipalities'].astype(int).sum()

df_totais['Has item information'] = ((tabela['Has item information'].astype(float)*tabela['Number of tenders'].astype(int)).sum())/(tabela['Number of tenders'].astype(int).sum())
df_totais['Has participant information'] = ((tabela['Has participant information'].astype(float)*tabela['Number of tenders'].astype(int)).sum())/(tabela['Number of tenders'].astype(int).sum())

df_totais['Avg. number of items per tender'] = tabela['Avg. number of items per tender'].astype(float).mean()
df_totais['Avg. number of participants per tender'] = tabela['Avg. number of participants per tender'].astype(float).mean()
df_totais['Avg. number of suppliers per tender'] = tabela['Avg. number of suppliers per tender'].astype(float).mean()

df_totais['Number of distinct participants'] = tabela['Number of distinct participants'].astype(int).sum()
df_totais['Number of distinct suppliers'] = tabela['Number of distinct suppliers'].astype(int).sum()
df_totais['Firms among suppliers'] = ((tabela['Firms among suppliers'].astype(float)*tabela['Number of distinct suppliers'].astype(int)).sum())/(tabela['Number of distinct suppliers'].astype(int).sum())

df_totais['Non-competitive tenders'] = ((tabela['Non-competitive tenders'].astype(float)*tabela['Number of tenders'].astype(int)).sum())/(tabela['Number of tenders'].astype(int).sum())
df_totais['Deserted tenders'] = ((tabela['Deserted tenders'].astype(float)*tabela['Number of tenders'].astype(int)).sum())/(tabela['Number of tenders'].astype(int).sum())
df_totais['Unsuccessful tenders'] = ((tabela['Unsuccessful tenders'].astype(float)*tabela['Number of tenders'].astype(int)).sum())/(tabela['Number of tenders'].astype(int).sum())
df_totais['Non-competitive tenders value'] = ((tabela['Non-competitive tenders value'].astype(float)*tabela['total_valor_corrigido'].astype(int)).sum())/(tabela['total_valor_corrigido'].astype(int).sum())

# Drop non-necessary variables

tabela = pd.concat([tabela,df_totais],axis=0)
tabela['sigla_uf'] = tabela['sigla_uf'].replace(np.nan,'Total')

tabela.drop(['Number of tenders with items','Number of tenders with participants',
             'firmas','Number of suppliers firms','non_competitive_tender', 'deserted_tender',
             'unsuccessful_tender','total_valor_corrigido'],axis=1,inplace=True)

#format
variables_list = ['Has item information','Has participant information',
                  'Firms among suppliers','Non-competitive tenders',
                  'Deserted tenders','Unsuccessful tenders',
                  'Non-competitive tenders value']

for var in variables_list:
    tabela[var] = tabela[var] * 100

tabela = tabela.reset_index()

tabela.to_csv(os.path.join(tab_output,'descriptive_statistics_procurement.csv'), index=False, na_rep='', float_format='%.2f')

tabela_pivo = tabela.transpose()
tabela_pivo = tabela_pivo.drop(tabela_pivo.index[0])
tabela_pivo = tabela_pivo.rename(columns=tabela_pivo.iloc[0])
tabela_pivo = tabela_pivo.drop(tabela_pivo.index[0])

ordem = ['Number of tenders','Deserted tenders','Non-competitive tenders','Non-competitive tenders value',
         'Unsuccessful tenders','Has item information','Avg. number of items per tender','Has participant information',
         'Number of distinct participants', 'Number of distinct suppliers','Firms among suppliers',
         'Avg. number of participants per tender','Avg. number of suppliers per tender','Number of distinct municipalities']

tabela_pivo = tabela_pivo.reindex(ordem)

# export TeX table
with open(os.path.join(tab_output,'descriptive_statistics.tex'), 'w') as f:
    f.write(tabulate(tabela_pivo, headers='keys', tablefmt='latex_booktabs', floatfmt=".2f", missingval=''))